Note

Regridding takes 1 second per panel when running with DASK as

- n_workers = 5 
- threads_per_worker = 4
- memory_limit = "30GiB" - memory per worker

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import importlib # for reloading imported modules without having to restart the kernel
import sys
# sys.path.append('/nobackup/phymku/WP5/WACCM-RR/analysis/maps_2/') # I shouldn't have to do this to read utils, but for some reason on arc4 I do
import utils
sys.path.append('../../helpers/') # '/nobackup/phymku/WP5/WACCM-RR/helpers/' | ../../helpers/
import calculate_rho_w # import convert_omega_to_w as omega_to_w
from scipy.interpolate import griddata

import time 
from dask.distributed import Client, LocalCluster

# Spin up Dask

In [2]:
# User changes - change and execute only if you want to run with Dask
n_workers = 5 
threads_per_worker = 4
memory_limit = "30GiB"
# End User Changes

cluster = LocalCluster(n_workers=5,threads_per_worker=4,memory_limit="30GiB")
client = Client(cluster)

client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mmkupilas/mmk_server/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mmkupilas/mmk_server/proxy/8787/status,Workers: 5
Total threads: 20,Total memory: 150.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43633,Workers: 5
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mmkupilas/mmk_server/proxy/8787/status,Total threads: 20
Started: Just now,Total memory: 150.00 GiB
Comm: tcp://127.0.0.1:45387,Total threads: 4
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mmkupilas/mmk_server/proxy/40577/status,Memory: 30.00 GiB
Nanny: tcp://127.0.0.1:43081,


In [ ]:
# # Clean up if needed
# client.shutdown()
# cluster.close() 

# User choices for reading in raw data, regridding and processing

In [3]:
# User changes - options to read in data that is regridded from scratch

# Path to files
# path = "/glade/derecho/scratch/mmkupilas/archive/f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.001/atm/hist/" # NR1
path = "/glade/derecho/scratch/mmkupilas/archive/f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.002/atm/hist/" # NR2
# path = "/resstore/b0243/Data/"



# NR2 files to read in
# file_list = ['f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.001.cam.h0i.2010-01-01-00900.nc',
#              'f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.001.cam.h0i.2010-01-02-00900.nc',
#              'f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.001.cam.h0i.2010-01-03-00900.nc'] 

# R1 files to read in
# file_list = ['f.e23.FWmaHIST.ne0np4.tromso01.ne30x8.L135.no_tromso_gw.cam.h0i.2010-02-19-00225.nc']
# file_list = ['temp2.nc']
# file_list = ['f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.001.cam.h0i.2010-02-27-00900.nc'] # NR1
file_list = ['f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.002.cam.h0i.2010-02-27-00900.nc'] # NR2

In [4]:
importlib.reload(utils) # don't touch - reloads utils

# ----------------------------------------------------------------
# User changes
# ----------------------------------------------------------------

# Choice of lon representation in dataset.
lon_range_type = 'n180_to_p180' # Options: 'O_to_360' ; 'n180_to_p180'

# Variables to extract from main dataset. Needs lat lon for regridding.
# Options: Any variable available from raw output in files from file_list.
# var_list_selection = ['T','U','V','O','NO','O3','OMEGA','ELECDEN','lat','lon']
var_list_selection = ['T','OMEGA','lat','lon']#'ELECDEN','U','V',]  

# Variables to actually regrid, i.e., you don't want lat or lon!
var_list_for_regridding = ['T','w']#,'T','ELECDEN','U','V'] 

# Calculate omega from T and OMEGA. If False, use OMEGA directly.
calculate_w_from_omega = True                 

# Pressure levels to index from full vertical column. 0 = model top, 125 = model bottom
# [3,15,43,103] chosen for paper
lev_list_selection = [1e-4]#,1e-3,1e-2,1e-1,1e0,1e1,1e2]

# Extraction of time from raw data read in.
itime_list_selection = np.arange(0,96,4) 

# Saves regridded file to netcdf
save_regridding = True

# Path to save regridded data, ignored in code if save_regridding = False
path_to_save_regrid = '/glade/derecho/scratch/mmkupilas/Analysis/MesoS2D/WP5/WACCM-RR/processed_data_2/maps/regrids/' 
# /glade/derecho/scratch/mmkupilas/Analysis/MesoS2D/WP5/WACCM-RR/processed_data_2/maps/regrids/
# /nobackup/phymku/WP5/WACCM-RR/processed_data_2/maps/regrids/

# String that goes at the front of model name when saving regridded file to netcdf
model_str = 'tempmmk_NR2' 

# Regridding extents (will just be ignored if regridding = False

# 2025-02-25 MMK: initial plots 
# rxL = -10 # 'regrid' x lower bound
# rxU = 50 # rx upper bound
# ryL = 50 
# ryU = 80
rxL = -45 # 'regrid' x lower bound
rxU = 80 # rx upper bound
ryL = 40 
ryU = 90
dr = 0.1 # Grid cell size

# Full domain
# rxL = -180 # 'regrid' x lower bound
# rxU = 180 # rx upper bound
# ryL = -90
# ryU = 90
# dr = 1 # Grid cell size

# ----------------------------------------------------------------
# End user changes
# ----------------------------------------------------------------


# Ingest and reduce data

In [5]:
# Read in files and append each to ds_list
ds_list = []
%time for file in file_list: ds_list.append(xr.open_mfdataset(path+file,parallel=True)[var_list_selection])

CPU times: user 109 ms, sys: 25 ms, total: 134 ms
Wall time: 819 ms


In [6]:
# Concat datasets from ds_list
%time ds = xr.concat(ds_list,dim='time')#.chunk({'ncol':20000,'time':100})

CPU times: user 4.17 ms, sys: 236 μs, total: 4.41 ms
Wall time: 3.59 ms


In [7]:
# Inspect dataset
print(f'ds takes up {ds.nbytes/1e9:.2f} Gb of memory')
print(ds)

ds takes up 5.11 Gb of memory
<xarray.Dataset> Size: 5GB
Dimensions:  (time: 96, lev: 135, ncol: 48600)
Coordinates:
  * lev      (lev) float64 1kB 6.14e-06 1.433e-05 2.788e-05 ... 991.2 997.5
  * time     (time) object 768B 2010-02-27 00:15:00 ... 2010-02-28 00:00:00
Dimensions without coordinates: ncol
Data variables:
    T        (time, lev, ncol) float32 3GB dask.array<chunksize=(96, 135, 48600), meta=np.ndarray>
    OMEGA    (time, lev, ncol) float32 3GB dask.array<chunksize=(96, 135, 48600), meta=np.ndarray>
    lat      (time, ncol) float64 37MB dask.array<chunksize=(96, 48600), meta=np.ndarray>
    lon      (time, ncol) float64 37MB dask.array<chunksize=(96, 48600), meta=np.ndarray>
Attributes:
    ne:                30
    fv_nphys:          3
    Conventions:       CF-1.0
    source:            CAM
    case:              f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.002
    logname:           mmkupilas
    host:              dec0071
    initial_file:      /glade/derec

In [8]:
# ds.lev.to_netcdf('lev_list.nc')

In [9]:
# Select data to regrid
%time ds0 = ds.isel(time=itime_list_selection).sel(lev=lev_list_selection,method='nearest') #.where((ds.lat.isel(time=0) > ryL) & (ds.lat.isel(time=0) < ryU) & (ds.lon.isel(time=0) > rxL) & (ds.lon .isel(time=0)< rxU))#.dropna(dim='ncol')

CPU times: user 2.87 ms, sys: 2.41 ms, total: 5.28 ms
Wall time: 4.89 ms


In [10]:
# Select choice of longitude axis from -180 to 180 or from 0 - 360 and sort out data
if lon_range_type == 'O_to_360':
    None
if lon_range_type == 'n180_to_p180':
    ds0['lon'] = xr.where(ds0['lon'] > 180, ds0['lon'] - 360, ds0['lon'])

In [11]:
print(ds0.nbytes/1e9)
print(ds0)

0.0279938
<xarray.Dataset> Size: 28MB
Dimensions:  (time: 24, lev: 1, ncol: 48600)
Coordinates:
  * lev      (lev) float64 8B 0.0001149
  * time     (time) object 192B 2010-02-27 00:15:00 ... 2010-02-27 23:15:00
Dimensions without coordinates: ncol
Data variables:
    T        (time, lev, ncol) float32 5MB dask.array<chunksize=(24, 1, 48600), meta=np.ndarray>
    OMEGA    (time, lev, ncol) float32 5MB dask.array<chunksize=(24, 1, 48600), meta=np.ndarray>
    lat      (time, ncol) float64 9MB dask.array<chunksize=(24, 48600), meta=np.ndarray>
    lon      (time, ncol) float64 9MB dask.array<chunksize=(24, 48600), meta=np.ndarray>
Attributes:
    ne:                30
    fv_nphys:          3
    Conventions:       CF-1.0
    source:            CAM
    case:              f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.002
    logname:           mmkupilas
    host:              dec0071
    initial_file:      /glade/derecho/scratch/mmkupilas/f.e23.FWmaHIST.ne30pg...
    topography_fi

In [12]:
# Load dataset into memory
# Takes ~ 1 min for a dataset with dims time: 384, lev: 135, ncol: 79274 with 33 Gb of data
# to be reduced to dataset with dims time: 96, lev: 4, ncol: 79274 with 360 Mb of data
# del(ds1)
%time ds1 = ds0.compute()
# %time ds1 = ds1.load()

CPU times: user 444 ms, sys: 89.7 ms, total: 534 ms
Wall time: 8.73 s


In [13]:
print(ds1.nbytes/1e9)
print(ds1)

0.0279938
<xarray.Dataset> Size: 28MB
Dimensions:  (time: 24, lev: 1, ncol: 48600)
Coordinates:
  * lev      (lev) float64 8B 0.0001149
  * time     (time) object 192B 2010-02-27 00:15:00 ... 2010-02-27 23:15:00
Dimensions without coordinates: ncol
Data variables:
    T        (time, lev, ncol) float32 5MB 231.3 227.4 222.6 ... 258.0 252.6
    OMEGA    (time, lev, ncol) float32 5MB -2.811e-07 -1.917e-07 ... 1.288e-06
    lat      (time, ncol) float64 9MB -35.03 -35.48 -35.92 ... 36.66 36.2 35.74
    lon      (time, ncol) float64 9MB -44.5 -43.5 -42.5 ... 137.0 136.0 135.0
Attributes:
    ne:                30
    fv_nphys:          3
    Conventions:       CF-1.0
    source:            CAM
    case:              f.e23.FWmaHIST.ne30pg3_ne30pg3_mg17_L135.no_tromso_gw.002
    logname:           mmkupilas
    host:              dec0071
    initial_file:      /glade/derecho/scratch/mmkupilas/f.e23.FWmaHIST.ne30pg...
    topography_file:   /glade/work/juliob/bndtopo/ne30pg3_gmted2010_modis_b

In [14]:
importlib.reload(calculate_rho_w) # Reloads function and updates any saved changes
if calculate_w_from_omega == True:
    ds1['w'] = calculate_rho_w.converta_omega_to_w(ds1['OMEGA'],ds1['T'],ds1['lev'])

# Regrid

In [15]:
# Get regridding geometry

# Extract dimension data
x=ds1['lon'].isel(time=0).data
y=ds1['lat'].isel(time=0).data

lat2d = np.arange(ryL, ryU + dr, dr) # create grid coordinates
lon2d = np.arange(rxL, rxU + dr, dr)
X, Y = np.meshgrid(lon2d, lat2d) # create grid
toc = time.time()

In [17]:
# ------------------------------------------------------------------------
# Perform regridding, if regrid_from_scratch FALSE, JUST ASSIGN ds2 TO ds_plot
# ------------------------------------------------------------------------
grand_tic = time.time()
for var in var_list_for_regridding:
    for ilev in range(len(lev_list_selection)):
        for itime in range(len(itime_list_selection)):
        # for itime in range(1):
            
            ds_for_regridding = ds1[var].isel(lev=ilev,time=itime)
            lev_str = f"{float(f'{ds_for_regridding.lev.values:.3g}'):.8f}" # Just a util for printing lev, 3 sig figs, 8 decimal places.
           
            print(f'processing var {var} lev {lev_str} hPa time {ds1.time.isel(time=itime).values}')

            regrid_tic = time.time()
            
                        
            regridded_data  = griddata((x,y), ds_for_regridding, (X, Y), method='linear') # regrid ds2
            ds_regridded = xr.Dataset({
                                    var: (["lat", "lon"], regridded_data),
                                },
                                coords={"lat": lat2d, "lon": lon2d})
            
            ds_regridded['time'] = ds_for_regridding.time # Add time information to dataset
            ds_regridded = ds_regridded.set_coords('time').expand_dims('time')
            ds_regridded['lev'] = ds_for_regridding.lev # Add lev information to dataset
            ds_regridded = ds_regridded.set_coords('lev').expand_dims('lev')
            regrid_toc = time.time()
            print(f'regridding took {regrid_toc-regrid_tic:.2f} seconds')
            
            # if save_regridding:
            #     fname = f'{model_str}_{var}_{lev_str}_{ds_regridded.time.values.item().strftime("%Y-%m-%d_%H:%M:%S")}.nc'
            #     print(f'Saving {path_to_save_regrid+fname}')
            #     ds_regridded.to_netcdf(path_to_save_regrid+fname)

grand_toc = time.time()
print(f'That whole cell took {grand_toc - grand_tic:.2f} seconds')

processing var T lev 0.00011500 hPa time 2010-02-27 00:15:00
regridding took 0.39 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 01:15:00
regridding took 0.38 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 02:15:00
regridding took 0.39 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 03:15:00
regridding took 0.38 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 04:15:00
regridding took 0.39 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 05:15:00
regridding took 0.38 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 06:15:00
regridding took 0.39 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 07:15:00
regridding took 0.38 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 08:15:00
regridding took 0.39 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 09:15:00
regridding took 0.38 seconds
processing var T lev 0.00011500 hPa time 2010-02-27 10:15:00
regridding took 0.39 seconds
processing

NameError: name 'grant_toc' is not defined

In [ ]:
y = ds.lat[0]
x = ds.lon[0]

plt.scatter(x,y,s=0.05)